In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
#gerekli kütüphaneler

In [ ]:
#CSV dosyasını okuyarak bir DataFrame oluşturuluyor.
df = pd.read_csv("bodyPerformance.csv")  

In [ ]:
df.isnull().sum() #Eksik değer kontrolü

In [ ]:
df.info()  #DataFrame bilgileri

In [ ]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
class_encoder = LabelEncoder() 
#LabelEncoder nesnesi oluşturuluyor.
#numerik olmayan değerler için encoding yapılacak

In [ ]:
df['gender'] = gender_encoder.fit_transform(df['gender']) # Female için 0 Male için 1 değeri 

# Burada sınıf birleştirme işlemi yapılmıştır daha iyi performans için B ve C sınıfı orta kategori olarak düşünülerek birleştirilmiştir
df['class_x'] = [0 if each == 'A' else 1 if each in ['B', 'C'] else 2 for each in df['class']]
# İşlem sonunda 4 farklı kategori 3 kategoriye indirgenmiştir bu işlem modelin performansını belirli ölçüde artırmıştır
df.head() 

In [ ]:
X = df.drop(["class_x","class"],axis=1) 
y = df.class_x  #df.class yazımı "invalid syntax" hatası verdiği için class yerine class_x ifadesi kullanıldı

In [ ]:
X.head() #girdiler

In [ ]:
y.head() #çıktılar

In [ ]:
from sklearn.model_selection import train_test_split #test ve eğitim verisini ayırabilmek için gerekli kütüphane

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , y , train_size=0.8, random_state=42) #test ve eğitim verisi oluşturma
#aynı sonuçları üretmek için random_state parametresi eklendi

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)  # veri setini kaç farklı şekilde ayıracağımızı belirledik
kf

In [ ]:
for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
    print(train_index, test_index)    #soldaki küme eğitim verisini içerir sağ taraf ise test verisini içerir

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.svm import SVC

# SVC modelini oluştur
svc_model1 = SVC(C=1, kernel='poly', degree=3)  #hiper parametre deneme yanılma ile seçildi
# SVC modelini eğitim verisi üzerinde eğit 
svc_model1.fit(X_train, y_train)         

# 5 kat çapraz doğrulama (cross-validation) ile doğruluk oranlarını hesapla
scores1 = cross_val_score(svc_model1, X, y, cv=5)

print("SVC Eğitim verisi doğruluk oranı:", svc_model1.score(X_train, y_train))
print("SVC KFold doğruluk oranı:", np.average(scores1))

In [ ]:
svc_model2 = SVC(C=0.1, kernel='linear', degree=4)
svc_model2.fit(X_train,y_train)

scores2 = cross_val_score(svc_model2,X,y,cv=5)

print("SVC Eğitim verisi doğruluk oranı:",svc_model2.score(X_train,y_train))
print("SVC KFold doğruluk oranı:",np.average(scores2))

In [ ]:
average_train_accuracy = np.mean([svc_model1.score(X_train, y_train), svc_model2.score(X_train, y_train)])
print("Ortalama Eğitim verisi doğruluk oranı:", average_train_accuracy)

average_test_accuracy = np.mean([np.average(scores1),np.average(scores2)])
print("Ortalama KFold doğruluk oranı:", average_test_accuracy)

In [ ]:
y_predicted = svc_model1.predict(X_test) #test verileri için tahmin edilen değerleri değişkene atama işlemi

In [ ]:
from sklearn.metrics import confusion_matrix
cm =confusion_matrix(y_test, y_predicted)  #tahmin ve gerçek değerleri içeren matris oluşturma
cm

In [ ]:
import seaborn as sn              #görsel olarak yazdırmak için gerekli kütüphane
import matplotlib.pyplot as plt 

In [ ]:
# Matplotlib kütüphanesini kullanarak bir 10x6 büyüklüğünde bir ısı haritası (heatmap) figürü oluşturuyoruz.
plt.figure(figsize=(10, 6))

# Seaborn kütüphanesini kullanarak bir ısı haritası oluşturuyoruz. 'cm' matrisini kullanıyoruz ve sayısal değerleri ekrana yazdırıyoruz.
sn.heatmap(cm, annot=True, fmt="d")

# x ekseni etiketi "tahmin" olarak ayarlanıyor.
plt.xlabel('Tahmin')

# y ekseni etiketi "gerçek değer" olarak ayarlanıyor.
plt.ylabel('Gerçek Değer')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))
#Precision (Hassasiyet):Modelin pozitif olarak tahmin ettiği örneklerin ne kadarının gerçekten pozitif olduğunu gösterir.
#Recall (Duyarlılık):Gerçek pozitif örneklerin model tarafından ne kadarının doğru bir şekilde tespit edildiğini gösterir.
#F1-score:Hassasiyet ve duyarlılığın harmonik ortalamasını ifade eder.
#Support:Her bir sınıf için gerçek etiketlenmiş örnek sayısını temsil eder. 

#Accuracy (Doğruluk):Modelin doğru tahmin ettiği toplam örnek sayısının, tüm örnek sayısına oranını gösterir.
#Macro Avg (Makro Ortalama):Farklı sınıflar arasındaki metriklerin makro ortalamasını ifade eder. Her sınıfa eşit ağırlık verilir.
#Weighted Avg (Ağırlıklı Ortalama):Farklı sınıflar arasındaki metriklerin ağırlıklı ortalamasını ifade eder. Sınıfın örnek sayısına göre ağırlıklandırılır

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model1 = LogisticRegression(solver='liblinear',multi_class='ovr',max_iter=10000)
logistic_model1.fit(X_train,y_train)

l_scores = cross_val_score(logistic_model1,X,y,cv=5)

print("Lojistik Regresyon Eğitim verisi doğruluk oranı:",logistic_model1.score(X_train,y_train))
print("Lojistik Regresyon KFold doğruluk oranı ortalaması:",np.average(l_scores))

In [ ]:
logistic_model2 = LogisticRegression(solver='sag', multi_class='auto',max_iter=10000)
logistic_model2.fit(X_train,y_train)

#l_scores_train2 = cross_val_score(Logisitc_model2,X_train,y_train,cv=5)
l_scores_2 = cross_val_score(logistic_model2,X_test,y_test,cv=5)

print("Lojistik Regresyon Eğitim verisi doğruluk oranı:",logistic_model2.score(X_train,y_train))
print("Lojistik Regresyon KFold doğruluk oranı:",np.average(l_scores_2))

In [ ]:
average_train_accuracy = np.mean([logistic_model1.score(X_train,y_train),logistic_model2.score(X_train,y_train)])
print("Ortalama Eğitim verisi doğruluk oranı:", average_train_accuracy)

average_test_accuracy = np.mean([np.average(l_scores),np.average(l_scores_2)])
print("Ortalama KFold doğruluk oranı:", average_test_accuracy)

In [ ]:
y_predicted = logistic_model2.predict(X_test)
cm =confusion_matrix(y_test, y_predicted)
cm
plt.figure(figsize=(10,6))
sn.heatmap(cm,annot=True,fmt="d")
plt.xlabel('tahmin')
plt.ylabel('gerçek değer')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))

In [ ]:
from sklearn.ensemble import RandomForestClassifier   # sınıflandırma için kütüphane ekleme

forest_model1 = RandomForestClassifier(n_estimators=120,max_depth=12,min_samples_split=4,min_samples_leaf=3)
forest_model1.fit(X_train,y_train)

f_scores_test = cross_val_score(forest_model1,X,y,cv=5)

print("Random Forest Eğitim verisi doğruluk oranı:",forest_model1.score(X_train,y_train))
print("Random Forest KFold doğruluk oranı:",np.average(f_scores_test))

In [ ]:
forest_model2 = RandomForestClassifier(n_estimators=50,min_samples_split=2,max_depth=20)
forest_model2.fit(X_train,y_train)

f_scores_test2 = cross_val_score(forest_model2,X,y,cv=5)

print("Random Forest Eğitim verisi doğruluk oranı:",forest_model2.score(X_train,y_train))
print("Random Forest KFold doğruluk oranı:",np.average(f_scores_test2))

In [ ]:
average_train_accuracy = np.mean([forest_model1.score(X_train,y_train),forest_model2.score(X_train,y_train)])
print("Ortalama Eğitim verisi doğruluk oranı:", average_train_accuracy)

average_test_accuracy = np.mean([np.average(f_scores_test),np.average(f_scores_test2)])
print("Ortalama KFold doğruluk oranı:", average_test_accuracy)

In [ ]:
forest_model3 = RandomForestClassifier(n_estimators=40,max_depth=8)
forest_model3.fit(X_train,y_train)

f_scores_test3 = cross_val_score(forest_model2,X,y,cv=5)

print("Random Forest Eğitim verisi doğruluk oranı:",forest_model3.score(X_train,y_train))
print("Random Forest KFold doğruluk oranı:",np.average(f_scores_test3))

In [ ]:
y_predicted = forest_model3.predict(X_test)
cm =confusion_matrix(y_test, y_predicted)
cm
plt.figure(figsize=(10,6))
sn.heatmap(cm,annot=True,fmt="d")
plt.xlabel('tahmin')
plt.ylabel('gerçek değer')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted))